# 魔法属性

- 无论人或事物往往都有不按套路出牌的情况，Python的类属性也是如此，存在着一些具有特殊含义的属性，详情如下：

## 1. `__doc__`

- 表示类的描述信息

In [3]:
class Foo(object):
    """这是一段类的文字描述"""
    def __init__(self):
        pass

f = Foo()
print(f.__doc__)

这是一段类的文字描述


## 2. `__module__ 和 __class__`

- `__module__` 表示当前操作的对象在那个模块
- `__class__` 表示当前操作的对象的类是什么

In [5]:
class Foo(object):
    def func(self):
        pass
    

f = Foo()
print(f.__class__)
print(f.__module__)

<class '__main__.Foo'>
__main__


## 3. `__init__`

- 初始化方法，通过类创建对象时，自动触发执行

## 4. `__del__`

- 当对象在内存中被释放时，自动触发执行。
- 注：此方法一般无须定义，因为Python是一门高级语言，程序员在使用时无需关心内存的分配和释放，因为此工作都是交给Python解释器来执行，所以，`__del__`的调用是由解释器在进行垃圾回收时自动触发执行的。

## 5. `__call__`

- 对象后面加括号，触发执行。
- 注：`__init__`方法的执行是由创建对象触发的，即：对象 = 类名() ；而对于 `__call__` 方法的执行是由对象后加括号触发的，即：对象() 或者 类()()

In [8]:
class Foo(object):
    def __call__(self):
        print("__call__")

f = Foo()

f()

__call__


## 6. `__dict__`

- 类或对象中的所有属性
- 类的实例属性属于对象；类中的类属性和方法等属于类，即：

In [12]:
class Foo(object):
    def __init__(self, name):
        self.name = name
    
    def get_name(self):
        return self.name
    
f = Foo('xxx')

print(f.__dict__)
print(Foo.__dict__)


{'name': 'xxx'}
{'__module__': '__main__', '__init__': <function Foo.__init__ at 0x000002098C2A0C80>, 'get_name': <function Foo.get_name at 0x000002098C2A0598>, '__dict__': <attribute '__dict__' of 'Foo' objects>, '__weakref__': <attribute '__weakref__' of 'Foo' objects>, '__doc__': None}


## 7. `__str__`

- 如果一个类中定义了__str__方法，那么在打印 对象 或者使用 对象作为参数 时，默认输出该方法的返回值。

In [16]:
class Foo:
    def __str__(self):
        return "foo"


f = Foo()

print(f)

a = "{}".format(f)

print(a)

foo
foo


## 8、`__getitem__、__setitem__、__delitem__`

- 用于索引操作，如字典。以上分别表示获取、设置、删除数据
- 可以将一个类变为一个字典操作

In [17]:
# -*- coding:utf-8 -*-

class Foo(object):

    def __getitem__(self, key):
        print('__getitem__', key)

    def __setitem__(self, key, value):
        print('__setitem__', key, value)

    def __delitem__(self, key):
        print('__delitem__', key)


obj = Foo()

result = obj['k1']      # 自动触发执行 __getitem__
obj['k2'] = 'laowang'   # 自动触发执行 __setitem__
del obj['k1']           # 自动触发执行 __delitem__

__getitem__ k1
__setitem__ k2 laowang
__delitem__ k1


## 9、`__getslice__、__setslice__、__delslice__`

- 该三个方法用于分片操作，如：列表
- 可以将一个类当作列表操作

In [22]:
class Foo(object):

    def __getitem__(self, index):
        if isinstance(index, slice):
            print("Get slice---------> start: %s, stop: %s, step: %s." \
                    % (str(index.start), str(index.stop), str(index.step)))

    def __setitem__(self, index, value):
        if isinstance(index, slice):
            print("Set slice---------> start: %s, stop: %s, step: %s." \
                    % (str(index.start), str(index.stop), str(index.step)))
            print("\tThe value is:", value)
        
    def __delitem__(self, index):
        if isinstance(index, slice):
            print("Delete slice------> start: %s, stop: %s, step: %s." \
                    % (str(index.start), str(index.stop), str(index.step)))


if __name__ == "__main__":
    obj = Foo()
    obj[-1:10]
    obj[-1:10:1] = [2,3,4,5]
    del obj[-1:10:2]

Get slice---------> start: -1, stop: 10, step: None.
Set slice---------> start: -1, stop: 10, step: 1.
	The value is: [2, 3, 4, 5]
Delete slice------> start: -1, stop: 10, step: 2.


## 10. `__new__`

- 这个方法我们一般很少定义，不过我们在一些开源框架中偶尔会遇到定义这个方法的类。实际上，这才是“真正的构造方法”，它会在对象实例化时第一个被调用，然后再调用__init__，它们的区别主要如下：

- `__new__`的第一个参数是cls，而`__init__`的第一个参数是self
- `__new__`返回值是一个实例,返回给`__init__`，而`__init__`没有任何返回值，只做初始化操作
- `__new__`由于是返回一个实例对象，所以它可以给所有实例进行统一的初始化操作
- `__new__` + `__init__` 实现了像 c++ 中构造函数的作用
- 由于`__new__`优先于`__init__`调用，且返回一个实例，所以我们可以利用这种特性，每次返回同一个实例来实现一个单例类：

In [27]:
class Singleton(object):
    """单例"""
    _instance = None
    def __new__(cls, *args, **kwargs):
        if not cls._instance:
            cls._instance = super(Singleton, cls).__new__(cls, *args, **kwargs)
        return cls._instance
class MySingleton(Singleton):
    pass


a = MySingleton()
b = MySingleton()

assert a is b  	# True

- 另外一种使用场景是当你需要继承内置类时，例如int、str、tuple，只能通过__new__来达到初始化数据的效果：

In [29]:
class g(float):
    """千克转克"""
    def __new__(cls, kg):
        return float.__new__(cls, kg * 2)
# 50千克转为克
a = g(50)
print(a) 	# 100
print(a + 100)	# 200, 由于继承了float，所以可以直接运算，非常方便！

100.0
200.0


- 除此之外，`__new__`比较多的应用场景是配合元类使用